<a href="https://colab.research.google.com/github/lavanyashukla/tick-tock/blob/main/LWM_1_1_Tick_Tock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install openai -qq
from openai import OpenAI
from google.colab import userdata
from collections import Counter
import pandas as pd
import random
import json
import os

In [ ]:
!git clone https://github.com/lavanyashukla/tick-tock.git

fatal: destination path 'tick-tock' already exists and is not an empty directory.


In [ ]:
# load the data
input_file = open('tick-tock/data/support_tickets.json')
data = json.load(input_file)

Steps:

- Navigate to the "Secrets" pane from the left navigation bar
- Add the OPENAI_API_KEY name and value
- Turn on the toggle of "Notebook access"

In [ ]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=userdata.get('OPENAI_API_KEY')
)

# get the tags
file_path = 'tick-tock/data/tags.txt'
with open(file_path, 'r') as file:
    tags = [line.strip() for line in file]
print(tags)

# get the priorities
file_path = 'tick-tock/data/priorities.txt'
with open(file_path, 'r') as file:
    priorities = [line.strip() for line in file]
print(priorities)

['zopim_offline_message', 'zopim_chat_missed', 'zopim_chat_ended', 'zopim_chat', 'zendesk', 'yolov5', 'xgboost', 'workspace_2', 'workspace', 'workplace_2', 'workplace', 'workaround', 'windows', 'weights__biases', 'webinar', 'weave', 'watch', 'wandblogger', 'wandbdb', 'wandb_offline', 'waiting_on_info', 'vulnerability', 'videos', 'video', 'verification', 'vega-plots', 'vega', 'username', 'user-management', 'user_management_2', 'user_management__move_projects_2', 'user_management__move_projects', 'user_management___delete_user_2', 'user_management___delete_user', 'user_management', 'upgrade', 'updated_jira', 'update_jira', 'unresponsive_user_2', 'unresponsive_user', 'ui', 'type_question', 'type_product_feedback', 'type_feature_request', 'type_bug', 'trial', 'tracked_hours', 'ticket_roulette_chat', 'testing_bump', 'test', 'tensorflow', 'tensorboard_2', 'tensorboard', 'teams', 'team-settings', 'team-management', 'team_management_2', 'team_management', 'team', 'tags', 'tables_component', 't

# Make a filtered dataset

In [ ]:
# Flatten all tags from all dictionaries into a single list
all_tags = [tag for d in data for tag in d.get('tags', [])]

# Use Counter to count occurrences of each tag
tag_counts = Counter(all_tags)

# List of specific tags you're interested in
specific_tags = ["charts", "sweeps", "reports", "tables"]

# Filter the tag_counts for specific tags and create a sorted list of tuples (tag, count)
sorted_specific_tag_counts = sorted(
    [(tag, tag_counts.get(tag, 0)) for tag in specific_tags],
    key=lambda x: x[1], reverse=True
)

# Convert to DataFrame for nicer display
df = pd.DataFrame(sorted_specific_tag_counts, columns=['Tag', 'Count'])

# Display the DataFrame
print(df)

       Tag  Count
0   charts    903
1   sweeps    678
2  reports    310
3   tables    280


In [ ]:
# Filtering process
desired_tags = ["cli", "charts", "backend", "sweeps", "reports", "tables"]
filtered_data = [d for d in data if any(tag in d.get('tags', []) for tag in desired_tags)]

print(len(filtered_data))

3772


In [ ]:
# Let's look at the data
show_elem = 4
print(json.dumps(filtered_data[show_elem], indent=4))

{
    "description": "Hi, I've noticed that when I run a training script through `wandb agent` , a fatal exception in the code is not printed out to the console. While when I run the same code directly with `python train.py` I see the exception in the command output. Looks like if the agent was suppressing the exception printout to the console. Have you heard about something like that?",
    "raw_subject": "[SDK] wandb agent fatal exception not printed",
    "subject": "[SDK] wandb agent fatal exception not printed",
    "priority": "urgent",
    "problem_id": null,
    "tags": [
        "component_cli",
        "enterprise_customer",
        "halp",
        "p0",
        "question",
        "sweeps"
    ],
    "id": 36380,
    "360043868271": "question",
    "360042457771": [
        "enterprise_customer"
    ],
    "360042775872": [
        "component_cli"
    ],
    "360042457751": "sweeps",
    "360044019192": null,
    "360041678631": "https://weightsandbiases.slack.com/archives/C

In [ ]:
count, score_priority, score_tags = 0, 0, 0
acc = {}

# Prompt Engineering

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content

In [ ]:
print(acc)

{}


In [ ]:
def basic_prompt():
  count, score_tags = 0, 0
  for element in filtered_data[100:110]:
      # print(element)
      ticket_text = element['description']

      # Prompt – Classify Class
      prompt = f"""
      Classify the text delimited by triple backticks into one of the following classes.
      Classes: {desired_tags}
      Text: ```{ticket_text}```
      Class: """

      response = get_completion(prompt)

      print("Prediction: "+response)

      if(response in element['tags']):
          score_tags += 1
          print("Correct")
          print([tag for tag in element['tags'] if tag in desired_tags])
      else:
          print("Incorrect")
          print([tag for tag in element['tags'] if tag in desired_tags])
      count += 1
      print()

  print("__________________")
  print(f"Priority Accuracy: {score_tags/count}")
  return score_tags/count
acc['basic_prompt'] = basic_prompt()

Prediction: N/A
Incorrect
['backend']

Prediction: backend
Correct
['backend']

Prediction: charts
Incorrect
['tables']

Prediction: backend
Incorrect
['cli']

Prediction: ['backend']
Incorrect
['backend']

Prediction: ['sweeps']
Incorrect
['sweeps']

Prediction: ['backend']
Incorrect
['reports']

Prediction: charts
Correct
['charts']

Prediction: sweeps
Correct
['sweeps']

Prediction: ['backend']
Incorrect
['charts', 'cli']

__________________
Priority Accuracy: 0.3


In [ ]:
def add_5_examples():
  count, score_tags = 0, 0
  for element in filtered_data[100:110]:
      # print(element)
      ticket_text = element['description']
      examples = "".join([
                     f"Text: ```{element['description']}```\nClass: {next((tag for tag in desired_tags if tag in element['tags']), 'No matching tag')}\n"
                     for i, element in enumerate(filtered_data[:5])
                 ])

      # Prompt – Classify Class
      prompt = f"""
      Classify the text delimited by triple backticks into one of the following classes.
      Classes: {desired_tags}

      {examples}

      Text: ```{ticket_text}```
      Class: """
      response = get_completion(prompt)

      # print("Prompt: "+prompt)
      print("Prediction: "+response)

      if(response in element['tags']):
          score_tags += 1
          print("Correct")
          print([tag for tag in element['tags'] if tag in desired_tags])
      else:
          print("Incorrect")
          print([tag for tag in element['tags'] if tag in desired_tags])
      count += 1
      print()

  print("__________________")
  print(f"Priority Accuracy: {score_tags/count}")
  return score_tags/count
acc['add_5_examples'] = add_5_examples()

Prediction: N/A
Incorrect
['backend']

Prediction: backend
Correct
['backend']

Prediction: charts
Incorrect
['tables']

Prediction: backend
Incorrect
['cli']

Prediction: N/A
Incorrect
['backend']

Prediction: reports
Incorrect
['sweeps']

Prediction: reports
Correct
['reports']

Prediction: charts
Correct
['charts']

Prediction: sweeps
Correct
['sweeps']

Prediction: reports
Incorrect
['charts', 'cli']

__________________
Priority Accuracy: 0.4


In [ ]:
def add_100_examples():
  count, score_tags = 0, 0
  for element in filtered_data[100:110]:
      # print(element)
      ticket_text = element['description']
      examples = "".join([
                     f"Text: ```{element['description']}```\nClass: {next((tag for tag in desired_tags if tag in element['tags']), 'No matching tag')}\n"
                     for i, element in enumerate(filtered_data[:5])
                 ])

      # Prompt – Classify Class
      prompt = f"""
      Classify the text delimited by triple backticks into one of the following classes.
      Classes: {desired_tags}

      {examples}

      Text: ```{ticket_text}```
      Class: """
      response = get_completion(prompt)

      # print("Prompt: "+prompt)
      print("Prediction: "+response)

      if(response in element['tags']):
          score_tags += 1
          print("Correct")
          print([tag for tag in element['tags'] if tag in desired_tags])
      else:
          print("Incorrect")
          print([tag for tag in element['tags'] if tag in desired_tags])
      count += 1
      print()

  print("__________________")
  print(f"Priority Accuracy: {score_tags/count}")
  return score_tags/count
acc['add_100_examples'] = add_100_examples()

Prediction: N/A
Incorrect
['backend']

Prediction: backend
Correct
['backend']

Prediction: charts
Incorrect
['tables']

Prediction: backend
Incorrect
['cli']

Prediction: reports
Incorrect
['backend']

Prediction: reports
Incorrect
['sweeps']

Prediction: reports
Correct
['reports']

Prediction: charts
Correct
['charts']

Prediction: sweeps
Correct
['sweeps']

Prediction: reports
Incorrect
['charts', 'cli']

__________________
Priority Accuracy: 0.4


In [ ]:
def describe_classes():
  count, score_tags = 0, 0
  for element in filtered_data[100:110]:
      # print(element)
      ticket_text = element['description']
      examples = "".join([
                     f"Text: ```{element['description']}```\nClass: {next((tag for tag in desired_tags if tag in element['tags']), 'No matching tag')}\n"
                     for i, element in enumerate(filtered_data[:5])
                 ])

      # Prompt – Classify Class
      prompt = f"""
      Given the following description for each class:
      charts: Log rich plots in a W&B workspace to visualize model performace over time. Log images, video, audio, bar charts, histograms and more.
      sweeps: Use W&B Sweeps to automate hyperparameter search and visualize rich, interactive experiment tracking. Pick from popular search methods such as Bayesian, grid search, and random to search the hyperparameter space. Scale and parallelize sweep across one or more machines.
      reports: Use W&B Reports to organize Runs, embed and automate visualizations, describe your findings, and share updates with collaborators. Easily export your report as a LaTeX zip file or convert the file to PDF.
      tables: Log a table in an artifact for each meaningful step of training to analyze model performance over training time

      Classify the text delimited by triple backticks into one of the following classes.
      Classes: {desired_tags}

      {examples}

      Text: ```{ticket_text}```
      Class: """
      response = get_completion(prompt)

      # print("Prompt: "+prompt)
      print("Prediction: "+response)

      if(response in element['tags']):
          score_tags += 1
          print("Correct")
          print([tag for tag in element['tags'] if tag in desired_tags])
      else:
          print("Incorrect")
          print([tag for tag in element['tags'] if tag in desired_tags])
      count += 1
      print()

  print("__________________")
  print(f"Priority Accuracy: {score_tags/count}")
  return score_tags/count
acc['describe_classes'] = describe_classes()

Prediction: cli
Incorrect
['backend']

Prediction: backend
Correct
['backend']

Prediction: charts
Incorrect
['tables']

Prediction: charts
Incorrect
['cli']

Prediction: backend
Correct
['backend']

Prediction: sweeps
Correct
['sweeps']

Prediction: charts
Incorrect
['reports']

Prediction: charts
Correct
['charts']

Prediction: sweeps
Correct
['sweeps']

Prediction: reports
Incorrect
['charts', 'cli']

__________________
Priority Accuracy: 0.5


In [ ]:
print(pd.DataFrame(list(acc.items()), columns=['Prompt', 'Accuracy']).to_string(index=False))

          Prompt  Accuracy
    basic_prompt       0.3
  add_5_examples       0.4
add_100_examples       0.4
describe_classes       0.5
